In [26]:
!pip install -q kaggle

In [27]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [28]:
!mkdir dog_dataset
%cd dog_dataset

mkdir: cannot create directory ‘dog_dataset’: File exists
/content/dog_dataset


In [29]:
!kaggle datasets list -s dogbreedidfromcomp

ref                                title                    size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------  ----------------------  -----  -------------------  -------------  ---------  ---------------  
catherinehorng/dogbreedidfromcomp  dog-breed-id-from-comp  691MB  2020-06-26 03:09:05           1451          5  0.1764706        


In [30]:
!kaggle datasets download catherinehorng/dogbreedidfromcomp
%cd ..

 96% 666M/691M [00:02<00:00, 282MB/s]
100% 691M/691M [00:03<00:00, 239MB/s]
/content


In [ ]:
!unzip dog_dataset/dogbreedidfromcomp.zip -d dog_dataset
!rm dog_dataset/dogbreedidfromcomp.zip
!rm dog_dataset/sample_submission.csv

In [43]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.preprocessing import image
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import tensorflow as tf
import tensorflow 

from tensorflow import keras

In [33]:
labels = pd.read_csv('dog_dataset/labels.csv')
print(labels.shape)
labels.head()

(10222, 2)


,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [34]:
labels = labels.drop_duplicates()

In [35]:
labels.duplicated().sum()

0

In [36]:
breeds = labels['breed']
breed_count = breeds.value_counts()

In [37]:
breed_count.duplicated().sum()

75

In [15]:
breeds = breeds.drop_duplicates()

In [16]:
breeds.duplicated().sum()

0

In [17]:
class_names = list(breeds[0:10])

In [18]:
class_names

['boston_bull',
 'dingo',
 'pekinese',
 'bluetick',
 'golden_retriever',
 'bedlington_terrier',
 'borzoi',
 'basenji',
 'scottish_deerhound',
 'shetland_sheepdog']

In [19]:
labels = labels[(labels['breed'].isin(class_names))]

In [20]:
labels = labels.reset_index()

In [21]:
X_data = np.zeros((len(labels), 224, 224, 3), dtype='float')
Y_data = label_binarize(labels['breed'], classes=class_names)

In [22]:
for i in tqdm(range(len(labels))):
  img = image.load_img("dog_dataset/train/%s.jpg" % labels['id'][i], target_size=(224, 224))
  img = image.img_to_array(img)
  x = np.expand_dims(img.copy(), axis = 0)
  X_data[i] = x/255.0

print(X_data.shape)
print(X_data.size)
print(Y_data.shape)
print(Y_data.size)

100%|██████████| 870/870 [00:03<00:00, 286.46it/s]

(870, 224, 224, 3)
130959360
(870, 10)
8700


In [39]:
from scipy.ndimage import filters
model = Sequential()

model.add(Conv2D(filters=  64, kernel_size=(5,5), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=  32, kernel_size=(3,3), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=  16, kernel_size=(7,7), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=  8, kernel_size=(5,5), activation='relu', kernel_regularizer='l2'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dense(len(class_names), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 64)      4864      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 110, 110, 64)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 108, 108, 32)      18464     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 48, 48, 16)        25104     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 24, 24, 16)      

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_data,Y_data, test_size=0.2, random_state=42)

In [46]:
history = model.fit(
    X_train,
    y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = (X_test, y_test),
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    )

Epoch 1/100
6/6 [==============================] - 1s 171ms/step - loss: 0.3918 - accuracy: 0.9928 - val_loss: 8.2103 - val_accuracy: 0.2126
Epoch 2/100
6/6 [==============================] - 1s 147ms/step - loss: 0.3761 - accuracy: 0.9957 - val_loss: 8.2091 - val_accuracy: 0.2184
Epoch 3/100
6/6 [==============================] - 1s 147ms/step - loss: 0.3616 - accuracy: 0.9986 - val_loss: 8.1830 - val_accuracy: 0.2241
Epoch 4/100
6/6 [==============================] - 1s 144ms/step - loss: 0.3545 - accuracy: 1.0000 - val_loss: 8.1543 - val_accuracy: 0.2069
Epoch 5/100
6/6 [==============================] - 1s 142ms/step - loss: 0.3452 - accuracy: 1.0000 - val_loss: 8.0616 - val_accuracy: 0.2011
Epoch 6/100
6/6 [==============================] - 1s 145ms/step - loss: 0.3383 - accuracy: 1.0000 - val_loss: 7.9082 - val_accuracy: 0.2011
Epoch 7/100
6/6 [==============================] - 1s 142ms/step - loss: 0.3317 - accuracy: 1.0000 - val_loss: 7.7088 - val_accuracy: 0.2011
Epoch 8/100
6

In [47]:
acc_test = model.evaluate(X_test, y_test)
print(acc_test)

6/6 [==============================] - 0s 16ms/step - loss: 5.9704 - accuracy: 0.2126
[5.970414161682129, 0.21264368295669556]
